In [1]:
import numpy as np 
import os, sys
import pandas as pd
import matplotlib.pyplot as plt 
import scipy
import scipy.io 
import nn
import tensorflow as tf
from tensorflow import keras

import deepchem


%matplotlib inline

print(tf.__version__)
#import nn


# --------------------------------------------
# Load data
# --------------------------------------------
seed = 3452
split = 2

mb = 25 # size of minibatch
hist = 0.1 # fraction of the history to be remembered 

np.random.seed(seed)
dataset = scipy.io.loadmat('qm7.mat')

# --------------------------------------------
# Extract training data
# --------------------------------------------
P = dataset['P'][list(range(0,split))+list(range(split+1,5))].flatten()
P_unflat = dataset['P'][range(0,split)+range(split+1,5)]
X = dataset['X'][P]
T = dataset['T'][0,P]

print(P_unflat.shape, P.shape, X.shape, T.shape)
print(len(P), len(X), len(T))


ImportError: No module named 'matplotlib'

In [ ]:
p = dataset['P'][range(0, 4)]
x = dataset['X'][p]
t = dataset['T'][0,p]
print(p.shape)
print(x.shape)
print(t.shape)

# Generate input and output data 

#I, O = nn.Input(x[:100]),nn.Output(t[:100])

test_x = x[:100]
test_t = t[:100] 


print(test_x.shape, test_t.shape)

print(x.shape, t.shape)

test_t.shape

In [ ]:


# step = 1.0
# noise = 1.0 
# maxp = 0 

# triuind = (np.arange(23)[:,np.newaxis] <= np.arange(23)[np.newaxis,:]).flatten()

# def expand(X):
#     Xexp = []
#     for i in range(X.shape[1]):
#         for k in np.arange(0,maxp[i]+step, step):
#             Xexp += [np.tanh((X[:,i]-k)/step)]
#     return np.array(Xexp).T

# def realize(X):
#     def _realize_(x):
#         inds = np.argsort(-(x**2).sum(axis=0)**.5+np.random.normal(0,noise,x[0].shape))
#         x = x[inds,:][:,inds]*1
#         x = x.flatten()[triuind]
#         return x
#     return np.array([_realize_(z) for z in X])


# for _ in range(10): 
#     maxp = np.maximum(maxp,realize(X).max(axis=0))

# X = expand(realize(X))
# nbout = X.shape[1]
# mean = X.mean(axis=0)
# std = (X - mean).std()

# X = (X-mean)/std

# print(X.shape)




In [ ]:
print(test_x[0])

In [ ]:
# Create the neural network

inputs = keras.Input(shape=(23,23))


x = keras.layers.Dense(400, activation=tf.nn.sigmoid)(inputs)
x = keras.layers.Dense(100, activation=tf.nn.sigmoid)(x)
outputs = keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='atom_model')

print(model.summary())


#

# model = keras.Sequential([
#     #keras.layers.Flatten(input_shape=(4, 1433)),
#     keras.layers.Dense(400, activation=tf.nn.sigmoid),
#     keras.layers.Dense(100, activation=tf.nn.sigmoid), 
#     keras.layers.Dense(1)
# ])



# # Compile the model 

sgd = keras.optimizers.SGD(lr=0.01)

model.compile(optimizer=sgd, 
              loss='mean_squared_error',
             metrics=['mae', 'mse'])




In [ ]:
model.fit(test_x, test_t, epochs=5, batch_size=25)

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('END.')

EPOCHS = 50

history = model.fit(
  test_x, test_t,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()